# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


## Шаг 2. Предобработка данных

### Обработка пропусков

Сначала посмотрю есть ли аномальные значения в столбце 'children'.

In [4]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Нашел:   
-1 Вероятно, означает, что не известно количество детей. Данных для замены нет, поэтому оставлю как есть.  
20 Вероятно, опечатка и на самом деле это два ребёнка. Проведу замену.

In [5]:
data['children'].replace(20, 2, inplace=True)

Ранее увидел, что в столбце 'days_employed' неверные данные, из изучу его:

In [6]:
data[data['days_employed'].isnull() & ~(data['total_income'].isnull())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Убедился, что пропущенные данные в 'days_employed' совпадают с пропущенными в 'total_income'.

In [7]:
data.sort_values(by='days_employed', ascending=False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба


In [8]:
data[data['days_employed']>0].sort_values(by='days_employed').head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20444,0,328728.720605,72,среднее,1,вдовец / вдова,2,F,пенсионер,0,96519.339647,покупка жилья для семьи
9328,2,328734.923996,41,высшее,0,женат / замужем,0,M,пенсионер,0,126997.497760,операции со своей недвижимостью
17782,0,328771.341387,56,среднее,1,женат / замужем,0,F,пенсионер,0,68648.047062,операции с коммерческой недвижимостью
14783,0,328795.726728,62,высшее,0,женат / замужем,0,F,пенсионер,0,79940.196752,на покупку своего автомобиля
7229,1,328827.345667,32,среднее,1,гражданский брак,1,F,пенсионер,0,122162.965695,сыграть свадьбу


In [9]:
data[data['days_employed'] > 0]['income_type'].unique()

array(['пенсионер', 'безработный'], dtype=object)

In [10]:
data[data['days_employed'] < 0]['income_type'].unique()

array(['сотрудник', 'компаньон', 'госслужащий', 'студент',
       'предприниматель', 'в декрете'], dtype=object)

Выяснил, что:
1. Пустые значения в 'days_employed' совпадают с пустыми в 'total_income'. Заменю их на 0 в 'days_employed', т.к. эти данные не связаны с тем что мы исследуем.
2. Положительные значения есть только у песнионеров и безработных, и отвечают они точно не за рабочий стаж. Возможно они связаны с размером песнии или пособия, в поставленных задачах это не важно. Заменю на 0.
3. Кол-во дней стажа отрицательно, вероятно при выгрузке данных добавился минус к дням. Надо заменить на положительные.

In [11]:
data.loc[data['days_employed'] > 0, 'days_employed'] = 0
data['days_employed'] = data['days_employed'].fillna(0)
data.loc[data['days_employed'] < 0, 'days_employed'] = data.loc[data['days_employed'] < 0, 'days_employed'].abs()

In [12]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Есть нулевые значения в 'dob_years', значащие, вероятно, что не известен возраст человека. Посмотрю можно ли заменить, после заполнения пропусков в 'total_income'.

In [13]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [14]:
data['education_id'].unique()

array([0, 1, 2, 3, 4])

В столбце 'education' слова в разных регистрах, заменю на нижний, пользуясь education_id.

In [15]:
def education_rename(val):
    if val == 0:
        return 'высшее'
    elif val == 1:
        return 'среднее'
    elif val == 2:
        return 'неоконченное высшее'
    elif val == 3:
        return 'начальное'
    elif val == 4:
        return 'ученая степень'
    return 'error'
data['education'] = data['education_id'].apply(education_rename)

In [16]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [17]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

In [18]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В столбцах 'family_status', 'family_status_id' пропусков нет. В 'gender' есть один пропуск - 'XNA', удалю его.

In [19]:
data.drop(data[data['gender']=='XNA'].index, inplace=True)
data.reset_index(drop=True, inplace=True)

In [20]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [21]:
data['debt'].unique()

array([0, 1])

С 'income_type' и 'debt' всё в порядке.  
Перехожу к заполнению пропусков в 'total_income':

In [22]:
data[data['total_income']<=0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [23]:
data[data['total_income']>=1000000]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1590,1,1283.436610,53,высшее,0,женат / замужем,0,F,компаньон,0,1.089120e+06,на покупку автомобиля
2224,1,4614.851849,42,высшее,0,женат / замужем,0,M,сотрудник,0,1.278623e+06,покупка жилья
4129,0,7992.968249,52,высшее,0,женат / замужем,0,F,компаньон,0,1.004476e+06,операции с коммерческой недвижимостью
4415,0,542.321356,45,среднее,1,гражданский брак,1,M,компаньон,1,1.030899e+06,профильное образование
7447,0,8626.777234,39,среднее,1,в разводе,3,F,сотрудник,0,1.240165e+06,покупка недвижимости
9156,0,65.163190,52,высшее,0,женат / замужем,0,M,компаньон,0,1.006468e+06,операции с жильем
9169,1,5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1.726276e+06,дополнительное образование
9871,2,3092.259823,56,высшее,0,женат / замужем,0,M,сотрудник,0,1.103455e+06,покупка жилья для семьи
10004,0,1979.388318,58,среднее,1,женат / замужем,0,M,сотрудник,0,1.097955e+06,покупка своего жилья
11070,1,1851.200013,36,высшее,0,гражданский брак,1,F,сотрудник,0,1.286281e+06,покупка коммерческой недвижимости


In [24]:
data_grouped_for_income = data.groupby(['education_id', 'gender', 'income_type'])['total_income'].median()
data_grouped_for_income

education_id  gender  income_type    
0             F       безработный        202722.511368
                      госслужащий        157841.438279
                      компаньон          189893.464621
                      пенсионер          142404.278858
                      предприниматель    499163.144947
                      сотрудник          153691.851098
              M       госслужащий        214706.640963
                      компаньон          233064.883613
                      пенсионер          157457.331331
                      предприниматель              NaN
                      сотрудник          192247.298639
                      студент             98201.625314
1             F       в декрете           53829.130729
                      госслужащий        125831.806597
                      компаньон          144881.784198
                      пенсионер          112790.975121
                      сотрудник          124127.090016
              M       безра

In [25]:
data.drop(data[(data['education_id'] == 0) & (data['gender'] == 'M') & (data['income_type'] == 'предприниматель')].index, inplace=True)
data.reset_index(drop=True, inplace=True)
data_grouped_for_income = data.groupby(['education_id', 'gender', 'income_type'])['total_income'].median()
def med_total_income(row):
    return data_grouped_for_income[row['education_id'], row['gender'], row['income_type']]
data.loc[data['total_income'].isnull(), 'total_income'] = data.loc[data['total_income'].isnull(),].apply(med_total_income, axis=1)

Заменю 0 в 'dob_years', если есть данные на того же человека:

In [27]:
data[data['dob_years'] == 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,0.000000,0,среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,0.000000,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,0.000000,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,0.000000,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,153691.851098,жилье
1898,0,0.000000,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


In [28]:
data[['children','education_id', 'family_status_id', 'gender', 'income_type', 'total_income']].duplicated().sum()

1955

In [29]:
data[['children', 'dob_years','education_id', 'family_status_id', 'gender', 'income_type', 'total_income']].duplicated().sum()

828

In [30]:
group_years = data.groupby(['children', 'education_id', 'family_status_id', 'gender','income_type','total_income'])['dob_years'].max()
def look_pass_dob_years(row):
    result = group_years[row['children'], row['education_id'], row['family_status_id'],row['gender'], row['income_type'],row['total_income']]
    return result
data['dob_years'] = data.apply(look_pass_dob_years, axis=1)
data[['children', 'dob_years','education_id', 'family_status_id', 'gender', 'income_type', 'total_income']].duplicated().sum()

1955

In [31]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Пропусков в 'purpose' нет.

**Вывод**

Были явные пропуски в 'days_emloyed' и 'total_income', и неявные в 'dob_years' и 'children':
  -  В 'days_employed' заменил на 0.  
  -  В 'total_income' заменил на медианные значение по группам ['education_id', 'gender', 'income_type'].  
  -  В 'dob_years' посмотрел, есть ли данные на того же человека в этой таблице, если есть - заменил, если нет - оставил 0.  
  -  В 'children' оставил как есть.  
  -  В 'gender' была одна запись c 'XNA', удалил.

### Замена типа данных

In [32]:
data['days_employed'] = data['days_employed'].astype('int64')
data['total_income'] = data['total_income'].astype('int64')

**Вывод**

В 'days_employed' и 'total_income' заменили вещ. тип на целочисленный методом astype.

### Обработка дубликатов

In [33]:
data.duplicated().sum()

770

In [34]:
data[(data.duplicated())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
322,0,0,71,среднее,1,гражданский брак,1,F,пенсионер,1,112790,сыграть свадьбу
627,1,0,46,высшее,0,гражданский брак,1,M,сотрудник,0,192247,сыграть свадьбу
681,0,0,65,высшее,0,женат / замужем,0,M,сотрудник,0,192247,покупка жилой недвижимости
1118,0,0,68,среднее,1,женат / замужем,0,M,сотрудник,0,156286,сделка с автомобилем
1237,0,0,70,среднее,1,женат / замужем,0,F,пенсионер,0,112790,операции с недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21413,0,0,70,среднее,1,женат / замужем,0,F,пенсионер,0,112790,операции с жильем
21461,1,0,46,высшее,0,гражданский брак,1,M,сотрудник,0,192247,на проведение свадьбы
21493,1,0,50,среднее,1,гражданский брак,1,F,сотрудник,0,124127,свадьба
21495,0,0,58,высшее,0,женат / замужем,0,F,компаньон,0,189893,строительство недвижимости


In [35]:
data[(data.duplicated()) & (data['days_employed'] != 0)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Дупликаты есть либо с изначально пропущенными значениями в 'days_employed' и 'total_income', либо у пенсионеров с безработными.

In [36]:
data = data.drop_duplicates().reset_index(drop=True)

Посмотрю у всех ли верно указаны данные в задолженности по кредитам:

In [37]:
data[['children', 'days_employed', 'dob_years','education_id','family_status_id','gender','income_type', 'total_income', 'purpose']].duplicated().sum()

84

In [38]:
data[['children', 'days_employed', 'dob_years','education_id','family_status_id','gender','income_type', 'total_income','debt','purpose']].duplicated().sum()

0

Нет, исправлю:

In [39]:
temp_group = data.groupby(['total_income','children', 'education_id', 'family_status_id', 'gender','income_type', 'purpose'])['debt'].max()
def look_error_debt(row):
    result = temp_group[row['total_income'], row['children'], row['education_id'], row['family_status_id'],
                        row['gender'], row['income_type'], row['purpose']]
    return result
data['debt'] = data.apply(look_error_debt, axis=1)

In [40]:
data[['children', 'dob_years','education_id','family_status_id','gender','income_type', 'total_income', 'purpose']].duplicated().sum()

85

In [41]:
data[['children', 'dob_years','education_id','family_status_id','gender','income_type', 'total_income', 'debt', 'purpose']].duplicated().sum()

85

In [42]:
data = data.drop_duplicates(subset = ['children', 'dob_years','education_id','family_status_id','gender','income_type', 'total_income', 'purpose']).reset_index(drop=True)

**Вывод**

Искал дупликаты с помощью методов: duplicated() и drop_duplicates().

### Лемматизация

In [43]:
from pymystem3 import Mystem
m = Mystem()
def lem_purpose(purpose):
    lemmas = m.lemmatize(purpose)
    if 'образование' in lemmas:
        return 'образование'
    elif ('жилье' in lemmas) or ('недвижимость' in lemmas):
        return 'недвижимость'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    return 'error'
data['purpose'] = data['purpose'].apply(lem_purpose)
data['purpose'].value_counts()

недвижимость    10426
автомобиль       4171
образование      3891
свадьба          2180
Name: purpose, dtype: int64

In [44]:
data[['children', 'dob_years','education_id', 'family_status_id', 'gender', 'income_type', 'total_income', 'purpose']].duplicated().sum()

880

In [45]:
data = data.drop_duplicates(subset = ['children', 'dob_years','education_id','family_status_id','gender','income_type', 'total_income', 'purpose']).reset_index(drop=True)

**Вывод**

Выделил 4 группы целей кредитов лемматизировав их, с помощью библиотеки pymystem3. Потом еще раз прошелся по дупликатам.

### Категоризация данных

Разобью данные на 4 группы по уровню дохода:

In [46]:
def income_group(income):
    if income <= 60000:
        return 'низкий'
    elif income <= 150000:
        return 'средний-низкий'
    elif income <= 250000:
        return 'средний-высокий'
    return 'высокий'
data['income_group'] = data['total_income'].apply(income_group)
data.groupby('income_group')['income_group'].count()

income_group
высокий            2812
низкий              806
средний-высокий    6616
средний-низкий     9554
Name: income_group, dtype: int64

In [47]:
clients_data = data[['children', 'days_employed', 'dob_years', 'education_id', 'family_status','family_status_id', 
                     'gender', 'income_type', 'debt','total_income', 'income_group']].drop_duplicates().reset_index(drop=True)

In [48]:
credit_log = data[['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose', 'income_group']]
credit_education_dict = data[['education_id', 'education']].drop_duplicates().sort_values(by='education_id').reset_index(drop=True)
credit_family_status_dict = data[['family_status_id', 'family_status']].drop_duplicates().sort_values(by='family_status_id').reset_index(drop=True)

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [49]:
df_peop_with_child = clients_data[clients_data['children'] > 0]
df_peop_wout_child = clients_data[clients_data['children'] == 0]
fract_debt_with_child = (df_peop_with_child['debt'] == 1).sum() / df_peop_with_child.shape[0]
fract_debt_wout_child = (df_peop_wout_child['debt'] == 1).sum() / df_peop_wout_child.shape[0]
print('Процент людей с детьми, имеющих долги по кредитам: {:.2%}'.format(fract_debt_with_child))
print('Процент людей без детей, имеющих долги по кредитам: {:.2%}'.format(fract_debt_wout_child))

Процент людей с детьми, имеющих долги по кредитам: 9.47%
Процент людей без детей, имеющих долги по кредитам: 7.58%


**Вывод**

Люди с детьми чаще пропускают платежи по кредитам.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [50]:
family_group = clients_data.groupby('family_status')['debt']
family_status_res = family_group.sum() / family_group.count() * 100
family_status_res

family_status
Не женат / не замужем    10.027206
в разводе                 7.162041
вдовец / вдова            6.696935
гражданский брак          9.411146
женат / замужем           7.625156
Name: debt, dtype: float64

**Вывод**

Давать кредит лучше вдовцам или людям в разводе, холостяки не возвращают долги.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [51]:
total_income_group = clients_data.groupby('income_group')['debt']
total_income_res = total_income_group.sum() / total_income_group.count() * 100
total_income_res

income_group
высокий            6.899004
низкий             6.079404
средний-высокий    8.521419
средний-низкий     8.585005
Name: debt, dtype: float64

**Вывод**

Люди с высоким или низким уровнем дохода одинаково хорошо возвращают деньги, а вот у середнячков проблемы.

- Как разные цели кредита влияют на его возврат в срок?

In [52]:
purpose_group = data.groupby('purpose')['debt']
purpose_res = purpose_group.sum() / purpose_group.count() * 100
purpose_res

purpose
автомобиль      9.515485
недвижимость    7.304050
образование     9.183948
свадьба         7.878788
Name: debt, dtype: float64

**Вывод**

Кредит на недвижимость сильно сложнее получить и условия жестче, чем в кредите на автомобиль и образование. А данные о задолженностям по кредитам на свадьбу совпадают с данными о возврате кредитов женатыми людьми.

## Шаг 4. Общий вывод

Люди с детьми имеют долги по кредитам в 9.47%, что хуже, чем люди без детей с 7.58%.  
Из данных по семейному положению следует:  
Самые надежные заёмщики это вдовцы / вдовы с 6.70 % невозвратов и разведённые с 7.16%. Проблем с кредитами больше всего у не женатых / не замужних людей с 10.03%.  
По уровню дохода самые надёжные заёмщики это люди с низким или высоким уровнем дохода, 6.08% и 6.90% просрочек соответственно. У людей с средне-низким и средне-высоким проценты такие: 8.59% и 8.52%.  
Люди берущие кредит на недвижимость и образование становятся должниками в 7.30% и 7.88%, что реже чем люди берущие кредит на образование и автомобиль: в 9.18% на образование и 9.52% на автомобиль.